In [68]:
import pandas as pd
import numpy as np

results = pd.read_excel("fourth_down_decision_making.xlsx")

results = results[
    (results["distance"] <= 10)
    & (
        results["field_position_intervals"].isin(
            [
                "1-5",
                "21-25",
                "6-10",
                "26-30",
                "41-45",
                "16-20",
                "31-35",
                "46-50",
                "11-15",
                "36-40",
            ]
        )
    )
    & (results["time_intervals"].isin(['61-64', '58-60', '55-58', '28-31', '25-28']))
    & (results['score_differential'] >= -14)
    & (results['score_differential'] <= 14)
]
results = results.drop(columns=['play_type', 'first_down_prob', 'xPa', 'suggested_decision'])
results.head()

,down,distance,time_intervals,field_position_intervals,score_differential,field_goal_xPa,punt_xPa,go_for_it_xPa
5,4,1,25-28,46-50,7,2.789123,-0.031372,-0.172582
56,4,3,55-58,16-20,1,2.789838,-0.024289,-0.024377
57,4,5,61-64,26-30,-14,1.787823,-0.263876,-4.568935
67,4,7,28-31,46-50,11,1.411734,-0.012009,-0.006309
196,4,8,25-28,16-20,10,1.925114,-0.035047,0.150994


In [69]:
short = results[(results["distance"] >= 1) & (results["distance"] <= 3)]

short_results = short.groupby(['field_position_intervals', 'score_differential'])[['field_goal_xPa', 'punt_xPa', 'go_for_it_xPa']].agg('median').reset_index()

conditions = [
    (short_results["field_goal_xPa"] > short_results["punt_xPa"])
    & (short_results["field_goal_xPa"] > short_results["go_for_it_xPa"]),
    (short_results["punt_xPa"] > short_results["field_goal_xPa"])
    & (short_results["punt_xPa"] > short_results["go_for_it_xPa"]),
    (short_results["go_for_it_xPa"] > short_results["field_goal_xPa"])
    & (short_results["go_for_it_xPa"] > short_results["punt_xPa"]),
]

values = ["Attempt FG", "Punt", "Go For It"]

short_results["suggested_decision"] = np.select(conditions, values)

short_behind = short_results[short_results['score_differential'] < 0]
short_ahead_or_tied = short_results[short_results['score_differential'] >= 0]

short_results.head()

,field_position_intervals,score_differential,field_goal_xPa,punt_xPa,go_for_it_xPa,suggested_decision
0,1-5,-14,2.403286,-0.000874,6.440000,Go For It
1,1-5,-13,2.590581,-0.001403,5.949698,Go For It
2,1-5,-7,2.914901,-0.288795,0.876944,Attempt FG
3,1-5,-6,2.929888,-0.074779,4.617831,Go For It
4,1-5,0,2.773326,-0.004997,5.928889,Go For It


In [70]:
mid = results[(results["distance"] >= 4) & (results["distance"] <= 6)]

mid_results = mid.groupby(['field_position_intervals', 'score_differential'])[['field_goal_xPa', 'punt_xPa', 'go_for_it_xPa']].agg('median').reset_index()

conditions = [
    (mid_results["field_goal_xPa"] > mid_results["punt_xPa"])
    & (mid_results["field_goal_xPa"] > mid_results["go_for_it_xPa"]),
    (mid_results["punt_xPa"] > mid_results["field_goal_xPa"])
    & (mid_results["punt_xPa"] > mid_results["go_for_it_xPa"]),
    (mid_results["go_for_it_xPa"] > mid_results["field_goal_xPa"])
    & (mid_results["go_for_it_xPa"] > mid_results["punt_xPa"]),
]

values = ["Attempt FG", "Punt", "Go For It"]

mid_results["suggested_decision"] = np.select(conditions, values)

mid_behind = mid_results[mid_results['score_differential'] < 0]
mid_ahead_or_tied = mid_results[mid_results['score_differential'] >= 0]

mid_results['suggested_decision'].unique()

array(['Go For It', 'Attempt FG'], dtype=object)

In [71]:
long = results[(results["distance"] >= 7) & (results["distance"] <= 10)]

long_results = long.groupby(['field_position_intervals', 'score_differential'])[['field_goal_xPa', 'punt_xPa', 'go_for_it_xPa']].agg('median').reset_index()

conditions = [
    (long_results["field_goal_xPa"] > long_results["punt_xPa"])
    & (long_results["field_goal_xPa"] > long_results["go_for_it_xPa"]),
    (long_results["punt_xPa"] > long_results["field_goal_xPa"])
    & (long_results["punt_xPa"] > long_results["go_for_it_xPa"]),
    (long_results["go_for_it_xPa"] > long_results["field_goal_xPa"])
    & (long_results["go_for_it_xPa"] > long_results["punt_xPa"]),
]

values = ["Attempt FG", "Punt", "Go For It"]

long_results["suggested_decision"] = np.select(conditions, values)

long_behind = long_results[long_results['score_differential'] < 0]
long_ahead_or_tied = long_results[long_results['score_differential'] >= 0]

long_results['suggested_decision'].unique()

array(['Go For It', 'Attempt FG'], dtype=object)

In [73]:
with pd.ExcelWriter('fourth_down_results.xlsx') as writer:
    short_behind.to_excel(writer, sheet_name='short_behind', index=False)
    short_ahead_or_tied.to_excel(writer, sheet_name='short_ahead_or_tied', index=False)
    mid_behind.to_excel(writer, sheet_name='mid_behind', index=False)
    mid_ahead_or_tied.to_excel(writer, sheet_name='mid_ahead_or_tied', index=False)
    long_behind.to_excel(writer, sheet_name='long_behind', index=False)
    long_ahead_or_tied.to_excel(writer, sheet_name='long_ahead_or_tied', index=False)